In [5]:
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload, MediaFileUpload
from googleapiclient.discovery import build
from google.oauth2 import service_account
import os
import tempfile
import csv




directorio_credenciales = "data-424019-28bfddebf741.json"

#### ingresar a google drive

In [2]:
# Cargar credenciales desde el archivo JSON descargado
credentials = service_account.Credentials.from_service_account_file(directorio_credenciales)

# Crear cliente para acceder a la API de Google Drive
drive_service = build('drive', 'v3', credentials=credentials)

# ID de la carpeta en Google Drive
folder_id = '1tZHS0BuJoSG3DsfSmxgY5Am3Llj24JPY'

# Obtener lista de archivos en la carpeta especificada
results = drive_service.files().list(q=f"'{folder_id}' in parents", fields='files(id, name)').execute()
files = results.get('files', [])

if not files:
    print('No se encontraron archivos en la carpeta.')
else:
    print('Archivos en la carpeta:')
    for file in files:
        print(f'{file.get("name")} ({file.get("id")})')

Archivos en la carpeta:
EndInvFINAL12312016_Clean.csv (1LZC95-7JDYt2xD1-4faQVhYiTtgtmDqZ)
BegInvFINAL12312016_Clean.csv (1luNiPmNEalyKb_cxyibBHyi9Ex1krwGa)
SalesFINAL12312016_Clean.csv (1C9BYLiRVENvtPYvXVjd4GC9xSeIV0WTW)
PurchasesFINAL12312016_Clean.csv (11FvZTyzBBpIkr-WOIO6ocfQURpkMw-Jr)
InvoicePurchases12312016_Clean.csv (1kR6vbb9ZUXowRfha6jMVhXnN8UJyj2mT)
2017PurchasePricesDec_Clean.csv (1ALrqNvbvnr8aTKNdcC1mEfSq65Cg9tI-)


#### cambio de datos nulos y actualizacion del archivo

In [6]:
# Directorio temporal para descargar y manipular archivos
temp_dir = tempfile.gettempdir()

# Función para obtener la marca de tiempo de un archivo en Google Drive
def obtener_marca_de_tiempo_google_drive(file_id):
    # Esta función debe ser implementada utilizando la API de Google Drive
    # Aquí se ha colocado solo un esquema básico de la función
    file_metadata = drive_service.files().get(fileId=file_id, fields='modifiedTime').execute()
    modified_time = file_metadata.get('modifiedTime')
    return modified_time

# Función para obtener la marca de tiempo de la última ejecución del script
def obtener_marca_de_tiempo_ultima_ejecucion():
    if os.path.exists("ultima_ejecucion.txt"):
        with open("ultima_ejecucion.txt", "r") as file:
            last_execution_time = file.read().strip()
            return last_execution_time
    else:
        # Si el archivo no existe, se asume que es la primera ejecución y se devuelve una marca de tiempo "0"
        return "0"

# Obtener la lista de archivos modificados mientras iteras sobre los archivos en la carpeta
modified_files = []
for file in files:
    file_id = file['id']
    modified_time = obtener_marca_de_tiempo_google_drive(file_id)
    last_execution_time = obtener_marca_de_tiempo_ultima_ejecucion()  # Obtener la marca de tiempo de la última ejecución
    if modified_time > last_execution_time:
        modified_files.append(file_id)

# Si no hay archivos modificados, detener el proceso
if not modified_files:
    print("No hay archivos modificados.")
    # Aquí puedes agregar cualquier acción adicional que necesites
else:
    # Ahora puedes usar la lista de archivos modificados en tu bucle
    for file_id in modified_files:
        try:
            # Obtener el nombre del archivo original
            file_metadata = drive_service.files().get(fileId=file_id, fields='name').execute()
            file_name = file_metadata.get('name')

            # Descargar el archivo original a la carpeta local temporalmente
            download_path = os.path.join(temp_dir, file_name)
            request = drive_service.files().get_media(fileId=file_id)
            with open(download_path, 'wb') as fh:
                downloader = MediaIoBaseDownload(fh, request)
                done = False
                while not done:
                    status, done = downloader.next_chunk()
                    print(f"Descargado {int(status.progress() * 100)}%.")

            # Rellenar datos nulos con "no definido"
            with open(download_path, newline='') as csvfile:
                reader = csv.reader(csvfile)
                rows = list(reader)
                for row_index, row in enumerate(rows):
                    for col_index, cell in enumerate(row):
                        if not cell:
                            rows[row_index][col_index] = "no definido"

            # Guardar los cambios en el mismo archivo CSV
            with open(download_path, 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerows(rows)

            print(f"Datos nulos rellenados para el archivo con ID: {file_id}")

            # Actualizar el archivo existente en Google Drive con los cambios
            media = MediaFileUpload(download_path, resumable=True)
            updated_file = drive_service.files().update(fileId=file_id, media_body=media).execute()
            print('Archivo actualizado: %s' % file_id)
        except Exception as e:
            print('Error en el proceso: %s' % str(e))


Descargado 100%.


KeyboardInterrupt: 